In [26]:
from dotenv import load_dotenv

# 토큰 정보 로드
load_dotenv()

True

In [27]:
from typing import Dict, TypedDict

from langchain_core.messages import BaseMessage
import base64


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [28]:
import json
import matplotlib.pyplot as plt
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import Image, display
from langchain_community.chat_models import ChatOllama
import zlib
    

### Nodes ###
def summarize_user_request(state):
    """
    Converts user request into a short summarized goal
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The current graph state
    """
  
    state_dict = state["keys"]
    input = state_dict["input"]
    framework = state_dict["framework"]

    prompt = PromptTemplate(
        template=
        """
        Converts user request into a short summarized goal.

        Example 1:
          input = "I need a website that lets users login and logout. It needs to look fancy and accept payments."
          OUTPUT = "build a website that handles users logging in and logging out and accepts payments"
        Example 2:
          input = "Create something that stores crypto price data in a database using supabase and retrieves prices on the frontend."
          OUTPUT = "build a website that fetches and stores crypto price data within a supabase setup including a frontend UI to fetch the data."

          
        YOUR TURN!
        input:{input}
        OUTPUT:?

        """,
        input_variables=["input"],
    )
    # LLM
    # llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    llm = ChatOllama(model="mistral:latest")

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    input = rag_chain.invoke({"input": input})
    return {
        "keys": {"input": input, "framework": framework}
    }


def make_project_scope(state):
    """
    Takes in a user request to build a website project description
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates project_scope key with project scope
    """
    state_dict = state["keys"]
    input = state_dict["input"]
    framework = state_dict["framework"]

    prompt = PromptTemplate(
        template=
        """
        Takes in a user request to build a website project description.

        Output: Prints an object response in the following format:
          {{
            "is_crud_required": bool, // true if site needs CRUD functionality
            "is_user_login_and_logout": bool // true if site needs users to be able to log in and log out
            "is_external_urls_required": bool // true if site needs to fetch data from third part providers
          }}

        Example 1:
          input = "I need a full stack website that accepts users and gets stock price data"
          prints:
          {{
            "is_crud_required": true
            "is_user_login_and_logout": true
            "is_external_urls_required": true
          }}
        Example 2:
          input = "I need a simple TODO app"
          prints:
          {{
            "is_crud_required": true
            "is_user_login_and_logout": false
            "is_external_urls_required": false
          }}
          
        YOUR TURN!
        input:{input}
        OUTPUT:?

        """,
        input_variables=["input"],
    )

    # LLM
    # llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    llm = ChatOllama(model="mistral:latest")

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    project_scope = rag_chain.invoke({"input": input})
    return {
        "keys": {"input": input, "framework": framework, "project_scope": project_scope}
    }

def make_backend_code(state):
    """
    Create a backend code based on the user input, framework and project scope.
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates backend_code key with generated backend code.
    """

    state_dict = state["keys"]
    input = state_dict["input"]
    framework = state_dict["framework"]
    project_scope = state_dict["project_scope"]

    prompt = PromptTemplate(
        template=
        """
        Create a backend code based on the user input, framework and project scope.

        user input: {input}
        framework: {framework}
        project_scope: {project_scope}

        1. CRUD Logic: Generate models and controllers (or equivalent) to handle Create, Read, Update, and Delete operations for the data entities defined in the project. This includes setting up database connections, defining schemas or models, and creating the necessary endpoints for these operations.
        2. Login/Logout Logic: Implement an authentication system that supports user login and logout. This will involve generating user models, authentication controllers, and middleware for session management or token-based authentication (e.g., JWT), ensuring secure handling of user credentials and sessions.
        3. External URLs Logic: Create functionality to interact with external APIs or web resources. This includes setting up HTTP clients, configuring request handlers, and integrating these capabilities into the application to fetch, send, or manipulate data from external sources.

        Proceed to outline the basic structure of the backend code. This includes setting up the project environment, defining models for database interactions, implementing authentication mechanisms, and integrating external APIs or URLs. The prompt will guide the model to generate code snippets that illustrate these functionalities, ensuring that they are scalable, secure, and adhere to best practices for backend development.
        The generated backend code will serve as a template or starting point for further customization and development, enabling the user to quickly scaffold their project with essential features and focus on adding specific functionalities or business logic as per their requirements.
          
        Self Evaluation:
          - Review if the generated code covers all aspects of the project scope.
          - Assess the code for adherence to the chosen framework's conventions and best practices.
          - Ensure the code is modular, easily understandable, and scalable.
          - Verify security measures, especially in user authentication and data manipulation, to prevent common vulnerabilities.
          - Do not just code comment. You should generate codes following your code comment.
        
        Based on the self-evaluation, refine the prompt or generated code to better meet the project requirements, improve code quality, or enhance security and performance.

        IMPORATNT: Print ONLY the code, nothing else.
        IMPORTANT: Write functions that make sense for the users request if required.
        """,
        input_variables=["input", "framework", "project_scope"],
    )
    # LLM
    # llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)
    llm = ChatOllama(model="mistral:latest")

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    backend_code = rag_chain.invoke({"input": input, "framework": framework, "project_scope": project_scope})
    return {
        "keys": {"input": input, "framework": framework, "project_scope": project_scope, "backend_code": backend_code}
    }

# def improve_backend_code(state):
#     """
#     Improve a backend code based on intital backend code, framework and project scope.
    
#     Args:
#         state (dict): The current graph state

#     Returns:
#         state (dict): Updates backend_code key with improved backend code.
#     """

#     state_dict = state["keys"]
#     input = state_dict["input"]
#     framework = state_dict["framework"]
#     project_scope = state_dict["project_scope"]

#     prompt = PromptTemplate(
#         template=
#         """
#         Improve a backend code based on intital backend code, framework and project scope.

#         user input: {input}
#         framework: {framework}
#         project_scope: {project_scope}

#         1. Bug Fixing and Minor Enhancements: Analyze the existing code to identify and resolve any bugs. This includes fixing syntax errors, logical errors, runtime exceptions, and addressing performance issues. Additionally, implement minor enhancements that improve functionality without significantly altering the existing code structure or introducing new features outside the original project scope.
#         2. Comprehensive Feature Implementation: Review the project specifications in detail to ensure that all backend functionalities requested in the spec are fully implemented. This involves cross-referencing the initial code against the project requirements, including CRUD operations, user login/logout functionalities, and external URL interactions. If any required feature is missing or incompletely implemented, develop and integrate the necessary code to fulfill these requirements comprehensively. This step ensures that the backend is feature-complete as per the spec, eliminating the need for future code additions.
#         3. Code-Only Output: Generate the improved code without any commentary or explanatory text. The output should consist solely of the refined code, including the fixes for identified bugs, code for the added minor functionalities, and implementations for any missing features as per the project spec. The code should be presented in a clean, organized manner, following the conventions and best practices of the chosen backend framework.

#         The model's task is to execute these steps meticulously, ensuring that the final code output is robust, fully functional, and aligns perfectly with the project's backend requirements. The model must prioritize code quality, security, and performance throughout this process, ensuring the backend is scalable and maintainable.

#         Self Evaluation:
#         - After generating the improved code, perform a thorough review to confirm that all bugs have been fixed, and the code now includes the minor enhancements identified in step 1.
#         - Verify that every feature outlined in the project spec is fully and correctly implemented as per step 2, ensuring the backend is complete and no additional code needs to be written in the future.
#         - Ensure that the output strictly contains the code with no commentary, adhering to the instruction in step 3.

#         This prompt guides the model to focus solely on delivering high-quality, functional backend code that meets or exceeds the project's specifications.

#         IMPORATNT: YOU JUST PRINT OUT CODE ONLY.
#         """,
#         input_variables=["input", "framework", "project_scope"],
#     )
#     # LLM
#     llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

#     # Chain
#     rag_chain = prompt | llm | StrOutputParser()

#     # Run
#     backend_code = rag_chain.invoke({"input": input, "framework": framework, "project_scope": project_scope})
#     return {
#         "keys": {"input": input, "framework": framework, "project_scope": project_scope, "backend_code": backend_code}
#     }

In [29]:
import pprint

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("summarize_user_request", summarize_user_request)  # summarize_user_request
workflow.add_node("make_project_scope", make_project_scope)  # make_project_scope
workflow.add_node("make_backend_code", make_backend_code)  # make_backend_code

# Build graph
workflow.set_entry_point("summarize_user_request")
workflow.add_edge("summarize_user_request", "make_project_scope")
workflow.add_edge("make_project_scope", "make_backend_code")
workflow.add_edge("make_backend_code", END)

# Compile
app = workflow.compile()

In [30]:
# Run
inputs = {"keys": {"input": "Create something that stores crypto price data in a database using docker-compose", "framework": "fastapi"}}
for output in app.stream(inputs, {"recursion_limit": 10}):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

# Final generation
pprint.pprint(value['keys']['backend_code'])

"Node 'summarize_user_request':"
'\n---\n'
"Node 'make_project_scope':"
'\n---\n'
"Node 'make_backend_code':"
'\n---\n'
"Node '__end__':"
'\n---\n'
(" Here's a basic outline of the backend code based on your input using "
 'FastAPI:  (Note that due to the text length limitation, I will provide you '
 'an outline instead)\n'
 '\n'
 '```python\n'
 'from fastapi import FastAPI, Depends, HTTPException\n'
 'from pydantic import BaseModel\n'
 'import requests\n'
 'from typing import List, Optional\n'
 'from sqlalchemy import create_engine, Column, Integer, Float, String, '
 'DateTime\n'
 'from sqlalchemy.ext.declarative import declarative_base\n'
 'from sqlalchemy.orm import sessionmaker\n'
 '\n'
 'app = FastAPI()\n'
 'Base = declarative_base()\n'
 'engine = create_engine("sqlite+pysqlite3:///crypto_prices.db")\n'
 'SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)\n'
 '\n'
 'class CryptoPrice(Base):\n'
 '    __tablename__ = "crypto_prices"\n'
 '\n'
 '    id = Colum